# Cədvəlin verilən sıra nömrələri əsasında bərabər aralıqlı hissələrə bölünməsi

Bu "notebook"u yaratmaqda məqsədim öz qarşıma çıxan bir problemi həll etmək idi. Əlimdə içində yüz minlərlə sətir olan bir cədvəl var idi. Mən həmin cədvəli ardıcıl identifikasiya nömrələri ilə hər biri 1000-lik aralığı əhatə edən kiçik cədvəllərə çevirməli idim. Məsələn, 1001-dən 2000-ə qədər İD-ləri olan bütün sətirlər bir cədvələ, 2001-3000 aralığındakı nömrələr digər cədvələ yazılmalı idi. Problem həm də ondan ibarət idi ki, İD-lər ardıcıl gəlmirdi. Məsələn 3015-dən sonrakı İD 3020 ola bilərdi, arada bir sıra İD-lər isə cədvəldə yox idi. Buna görə də 1000-lik aralıqlı cədvəllərdə say bərabər olmayacaqdı. Yəni cədvəli İD-yə görə sıralayıb, sonra 1000-1000 bölə bilməzdim. Bu zaman cədvəllər mənim istədiyim kimi 3001-4000 kimi olmazdı, arada istifadə olunamayan İD-lərə görə, 1000 ədəd götürsəm axırıncı ədəd 4000-dən böyük olardı və siyahə səliqəli bölünməzdi. Ona görə aşağıdakı kimi həll yolu fikirləşdim. Bəlkə də, bu bir o qədər çox rast gəlinən problem deyil, yaxud da çox sadədir. Amma düşündüm ki məndən başqa da kiminsə işinə yaraya bilər. Odur ki, bölüşmək qərarına gəldim.

İlk olaraq lazımi paketləri yükləyirik. Edəcəyimiz iş sadəcə əlimizdəki cədvəli hissələrə bölmək olacağı üçün sadəcə "Pandas" paketi kifayətdir. "Numpy" paketini isə "list" formatındakı ddəyişənləri "print()" edərkən ekranda daha yığcam çap olunmasını təmin etmək üçün istifadə edəcəyik.

In [1]:
import pandas as pd
import numpy as np

Hissələrə bölmək istədiyimiz cədvəli "notebook"a yükləməliyik. Bunun üçün faylın yerləşdiyi "directory"yə, faylın adına və "pd.read_excel" funksiyasına ehtiyacımız var.

In [2]:
# read_path = 'C:\\Users\\...\\'
# read_file_name = "random.xlsx"
# bina_az_extr_df = pd.read_excel(read_path+read_file_name)
# bina_az_extr_df.head()

Hazırda isə biz "notebook"un hər kəs üçün rahat icra edilə bilməsindən ötrü özümüz "Pandas"ın köməyilə cədvəl yaradayağıq. Mən ID üçün aralığı 1 - 9000 arası götürdüm. Siz bu aralığı dəyişə bilərsiniz.

In [3]:
# 1) random ids grouped by 1000
step_size = 1000
first = 1
last = first + step_size
num_of_elements = 1000
iter = 0
id_list = []
while last <= 11_000:
  id_list.extend(np.random.randint(first,last,num_of_elements))
  # print(iter)
  # print(last)
  iter = iter + 1
  first = last
  last = last + step_size

# 2) remove duplicates
id_series = pd.Series(id_list).drop_duplicates()

# 3) add random characteristics

# is employed
is_emp_series = np.random.choice(['yes', 'no'],len(id_series), [0.4, 0.6]) 

# education level
educat_series  = np.random.choice(['1st', '2nd', '3rd', '4th'],len(id_series), [0.1, 0.3, 0.4, 0.2]) 

# salary satisfaction
sal_sat_series = np.random.choice(['1', '2', '3', '4', '5'],len(id_series), [0.1, 0.3, 0.3, 0.2, 0.1])


# 4) build a DataFrame
random_df = pd.DataFrame({'ID': id_series,
                          'is_employed': is_emp_series,
                          'education_level': educat_series,
                          'salary_satisfaction': sal_sat_series}) 

# 5) shuffle it
random_df = random_df.sample(frac = 1).reset_index(drop = True)

# result
random_df.head()

,ID,is_employed,education_level,salary_satisfaction
0,5145,no,2nd,4
1,7814,no,2nd,1
2,1554,yes,2nd,4
3,6807,no,2nd,4
4,1233,no,3rd,4


Aşağıda yeni bir funksiya yaradırıq. Bu funskiyanın məqsədi 

In [19]:
def group_ids_in_lists(df, id_col, size_of_list, list_name_pref = 'list_'):
  '''
  df - DataFrame you want to split;

  id_col - the ID column that you want to split;

  size_of_list = the number of ids in each list;

  list_name_pref - the prefix to add to the new created id lists

  ==============================================================

  Function returns lists of names of id lists.
  '''
  # create a sorted list of original IDs
  orig_id_list = sorted(list(df[id_col]))

  # creating lists for equally binned IDs

  # creating a list for range of numbers between the min and max value of original ID list
  start_id = orig_id_list[0]
  end_id = orig_id_list[-1] 
  number_list = [*range(start_id,end_id+1)]

  # creating equally binned lists from the sequential range of numbers in 'number_list'

  # number of elements in subsequent list
  steps = size_of_list

  # beginning index for list
  start_index = 0

  # ending index of splitted list
  split_end_index = size_of_list

  # end index of the whole 'number_list'
  end_index = len(number_list) - 1

  # iteration number | also is used to name new created list
  i = 1

  # first splitted list is created out of loop
  globals()[list_name_pref+str(i)] = [*range(number_list[start_index], number_list[split_end_index])]
  list_of_list_names = [list_name_pref+str(i)]

  while split_end_index < end_index:
      i = i + 1
      start_index = split_end_index

      # if reached to the final list
      if start_index + steps >= end_index:
          # the last list comes up to the last index in 'number_list'
          split_end_index = end_index
          globals()[list_name_pref+str(i)] = [*range(number_list[start_index], number_list[split_end_index]+1)]
          
      # if not reached to the final list    
      else:
          # go to the next part of ids
          split_end_index = start_index + steps
          globals()[list_name_pref+str(i)] = [*range(number_list[start_index], number_list[split_end_index])]

      #append list names to a list for further reference
      list_of_list_names.append(list_name_pref+str(i))

  # the function returns the list of names of id lists
  return list_of_list_names

In [20]:
id_lists = group_ids_in_lists(random_df, 'ID', 1000, 'id_list_')
np.array(id_lists)

array(['id_list_1', 'id_list_2', 'id_list_3', 'id_list_4', 'id_list_5',
       'id_list_6', 'id_list_7', 'id_list_8', 'id_list_9', 'id_list_10'],
      dtype='<U10')

In [22]:
for l in id_lists:
    print("List name:"  , l)
    print("Start ID:"   , globals()[l][0])
    print("End ID:"     , globals()[l][-1])
    print("="*100)
    #print(globals()['list_'+str(i)])

List name: id_list_1
Start ID: 3
End ID: 1002
List name: id_list_2
Start ID: 1003
End ID: 2002
List name: id_list_3
Start ID: 2003
End ID: 3002
List name: id_list_4
Start ID: 3003
End ID: 4002
List name: id_list_5
Start ID: 4003
End ID: 5002
List name: id_list_6
Start ID: 5003
End ID: 6002
List name: id_list_7
Start ID: 6003
End ID: 7002
List name: id_list_8
Start ID: 7003
End ID: 8002
List name: id_list_9
Start ID: 8003
End ID: 9002
List name: id_list_10
Start ID: 9003
End ID: 9999


Yaratdığımız siyahıda orijinal siyahıdakı İD-lərdən fərqli olaraq İD nömrələr ardıcıl gəlir. Yəni biz "range()" vasitəsilə siyahı yaratdığımız üçün verilən ilk və son ədəd arasındakı bütün ədədləri siyahıya daxil edirik. Amma qeyd etmişdik ki, orijinal siyahıda bəzi İD-lər qeyd olunmayıb. Bu fərq rosesdə heç bir problem yaratmayacaq. Çünki biz sonra müqayisə aparıb həm orijinal siyahıda, həm də bizim yaratdığımız 1000-lik aralıqlı siyahılarda olan ədədlər əsasında bölgü aparacağıq.

Aşağıda yaratdığımız funksiya verilən "list_of_lists" listindəki list adlarına əsasən "globals()" vasitəsilə həmin adda olan siyahıları tapacaq və onların içindəki ədədləri orijinal dataframedəki ədədlərlə müqayisə edərək eyni 1000-lik qrupa uyğun sətirləri bir dataframedə birləşdirəcək.

In [24]:
def split_dfs(df, id_col, list_of_lists, list_name_pref, df_name_pref = 'df_'):
  '''
  df - DataFrame you want to split;

  id_col - the ID column that you want to split;

  list_of_lists - the list consisting of names of given id lists;

  list_name_pref - the prefix of given id lists;

  df_name_pref - the prefix to add to the new created id dfs

  ==========================================================

  Function returns lists of df names.
  '''
  list_of_dfs = []
  for lst in list_of_lists:
      globals()[df_name_pref + lst.lstrip(list_name_pref)] = \
           df[[id_num in globals()[lst] for id_num in df[id_col]]]
      # print(df_name_pref + lst.lstrip(list_name_pref))

      # append name of created df to list for further reference
      list_of_dfs.append(df_name_pref + lst.lstrip(list_name_pref))
  
  # function returns lists of df names
  return list_of_dfs

In [25]:
np.array(split_dfs(random_df, 'ID', id_lists, 'id_list_'))

array(['df_1', 'df_2', 'df_3', 'df_4', 'df_5', 'df_6', 'df_7', 'df_8',
       'df_9', 'df_10'], dtype='<U5')

Daha sonra bu iki "group_ids_in_lists" və "split_dfs" funksiyalarını bir funksiyada birləşdiririk.

In [84]:
def split_df_on_id(df, id_col, size_of_df, list_name_pref = 'list_', df_name_pref = 'df_'): 
  '''
  df - DataFrame you want to split;

  id_col - the ID column that you want to split;

  size_of_df = the number of ids in each dataframe;
  
  list_name_pref - the prefix of given id lists;

  df_name_pref - the prefix to add to the new created id dfs

  ==========================================================

  Function returns lists of df names.

  '''

  # create list of ids
  id_lists = group_ids_in_lists(df, id_col, size_of_list=size_of_df, list_name_pref = list_name_pref)

  # split dataframes ased on id lists
  return split_dfs(df, id_col, list_name_pref = list_name_pref, df_name_pref = df_name_pref, list_of_lists = id_lists)

In [87]:
df_list = split_df_on_id(random_df, id_col = 'ID', size_of_df = 1000, list_name_pref = 'id_list_', df_name_pref = 'df_')
np.array(df_list)

array(['df_0', 'df_1', 'df_2', 'df_3', 'df_4', 'df_5', 'df_6', 'df_7',
       'df_8', 'df_9'], dtype='<U4')

Sonda yaratdığımız xüsusi "write_dfs" funksiyası ilə içində cədvəllərin adları olan siyahıdakı bütün cədvəlləri fayla çevirib qeyd etdiyimiz "directory"də yadda saxlayırıq.

In [13]:
# write_path = path = 'C:\\Users\\...\\'
# to_file = ".csv"
# file_name_pref = "bina_az_ext_" 

def write_dfs(df_list, write_path, file_name_pref, encode, to_file = '.csv'):
  '''
  df_list - list of given names of dataframes to convert to file

  write_path - the folder directory where we want to save files

  file_name_pref - the file name prefix we want to use 
                   as the same pattern for all the created files

  encode - used for recognition to characters in different languages 
           for Azerbaijani use: 'utf-8-sig'
           
  to_file - the file format that we want to create 
            ('.csv,' '.xlsx', '.pkl', etc.)

  ============================================================================

  Function creates and writes files to directory and does not return any value.
  '''
  for df in df_list:
      start = globals()[df][0]
      end = globals()[df][-1]
      globals()[df].to_csv(path + 
                          file_name_beg +
                          str(start) +
                          "_" +
                          str(end) +
                          file_type, encoding = encode)

In [ ]:
# use function
# write_dfs(df_list, write_path, file_name_pref, encode, to_file = '.csv')

Ən sonda faylı "directory"-dən oxuyan, yuxarıda bəhs etdiyimiz üsulla hissələrə bölən və ən sonda da verilən yeni "directory"də yadda saxlayan bir funksiya yaradırıq.

In [14]:
def split_file_on_ids(read_path, id_col, write_path, file_name_pref, encode, read_type = 'csv', to_file = '.csv'):
  '''
  read_path - the file name and directory we want to read;

  id_col - the column of ID we want to split the table on;

  write_path - the folder directory where we want to save files;

  file_name_pref - the file name prefix we want to use 
                   as the same pattern for all the created files;

  encode - used for recognition to characters in different languages
           while writing dataframe to a file.
           For Azerbaijani use: 'utf-8-sig';

  read_type - the file format to read
           
  to_file - the file format that we want to create: 
            ('.csv,' '.xlsx', '.pkl', etc.)

  ============================================================================

  Function creates and writes files to directory
  and return "Succesfully written to given path!" 
  in case of successful write result.
  '''
  # read file from directory
  if read_type == 'csv':
    df = pd.read_csv(read_path)
  elif read_type == 'excel':
    df = pd.read_csv(read_path)
  elif read_type == 'pickle':
    df = pd.read_pickle(read_path)
  else:
    df = pd.read_tale(read_path)

  # split dataframe to grouped dataframes based on id column
  df_lists = split_df_on_id(df, id_col, list_name_pref = 'list_', df_name_pref = 'df_')

  # write dataframes to directory as a file in defined format
  write_dfs(df_list, write_path, file_name_pref, encode, to_file = '.csv')
  print("Succesfully written to given path!")

In [ ]:
# use function
# split_file_on_ids(read_path, id_col, write_path, file_name_pref, encode, read_type = 'csv', to_file = '.csv')

Siz də "notebook"u endirib funksiyaları öz cədvəllərinizə tətbiq edə bilərsiniz. Ümid edirəm ki, hazırladığım kodu izah edə bildim. Yaranan hər hansısa bir sual olarsa, əlaqə saxlamaqdan çəkinməyin:

https://www.challengersdeep.com/